# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [20]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [21]:
file = pd.read_csv("../output_data/cities.csv")
file.dropna()
file.head()

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Temperature,Wind Speed
0,0,Atuona,43,PF,1651163430,78,-9.8000,-139.0333,78.44,21.63
1,1,Joshīmath,84,IN,1651163431,69,30.5667,79.5667,60.73,7.27
2,2,Lavrentiya,99,RU,1651163431,92,65.5833,-171.0000,26.17,9.08
3,3,Busselton,82,AU,1651163431,77,-33.6500,115.3333,56.89,5.53
4,4,Erenhot,100,CN,1651163432,21,43.6475,111.9767,52.30,6.89


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [22]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [23]:
# Store 'Lat' and 'Lng' into  locations
locations = file[["Lat", "Lng"]]
humidity = file["Humidity"].astype(float)

In [27]:
# Plot Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

#Create and add heat layer 
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
fig.add_layer(heatmap_layer)
#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [28]:
temp_file=file.loc[(file['Temperature']>=70)&(file['Temperature']<=80)]
wind_file=temp_file.loc[(temp_file['Wind Speed']<=10)]
ideal_weather=wind_file.loc[(wind_file['Cloudiness']==0)]
ideal_weather.head()

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Temperature,Wind Speed
35,35,Tupelo,0,US,1651163443,37,34.2576,-88.7034,73.22,5.75
55,55,Russell,0,US,1651163450,38,32.3502,-85.1999,78.08,8.05
128,128,São Filipe,0,CV,1651163476,71,14.8961,-24.4956,71.98,9.73
226,226,Kiryat Gat,0,IL,1651163514,44,31.6100,34.7642,73.40,6.82
270,270,Saint George,0,US,1651163273,18,37.1041,-113.5841,76.24,5.01


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [29]:
hotel_df = ideal_weather.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
    # get lat, lng 
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

{
    "html_attributions": [],
    "next_page_token": "Aap_uEA9qcTGTckL7ZhVRRicpwFzGVNUmq2HTw3TAND6gmhOXp7p8VubFIHctirAFO58v3MyvvyvT0UVGbFSW0quWeEFrmGYpU0BJ_UG25WRY8GewbYcSEiJ4k_XBNQJE3Qen9M89PJy4UeGpszz3YG52JUlSGeK8C4jvMs8_ezwxBLxIkQl7oYYm7Lt46uMgOM8kPMwAhpl_ry3UvWjnvfJJtBD_9fcjL4-5kfPJQuS4grF5XUo5_Lef4ZSS6b53qOamva73QjU2E_tOXz_psPfQQNwC_z7N94XzdtoBvru6oygK3aMTfNUs_3jLmQwpyZsiPXuq9R4714t9m1PHPoxvSaonYtVo2hgHRl8lyMDRTHhVq2xepy3UJXKY0gxEVqVUnLXdkWrJl0nsuvVv_dZhZlYIzhMvB-EZK_6c0ewGcrclEeFRl4-81uxqarX-MwG",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 34.2503537,
                    "lng": -88.4029695
                },
                "viewport": {
                    "northeast": {
                        "lat": 34.25186582989272,
                        "lng": -88.40173432010728
                    },
                    "southwest": {
                        "lat": 34.2491661

{
    "html_attributions": [],
    "next_page_token": "Aap_uEApzhedVMyojCj4h10LftoG038tCyxWhvXG_zAqJ2jBa9-9cW1KbYu3mg5UOkxE8z4UxOdDziL6tkpaJgu-AEGDq83anryig5gE6cyTYNqJphzhQwRhg8EDLrIuJ95_EPhk_fRnC2zGIAzGRmRDCwHsfijEBLSlUivD1q0BKbwnPh0_Z7YFAZj3CBI3Nfdc-XsdkOtLpf8LHHzNDHW34hXrov1ojtQf8oTHHNqXtOFb57zqkG-zntxxHleonXd1Gt5JEuEBqje5-QWnrU6av26njHCIfqgpm3tY9peltusLPhLwU1tn9Gkd7fVU_c-cWkdO3su9RcsbkVJhVbK5BILz9TlTgu4me70rn-aUW1X5kUbhON4KOgG2z7Clhg36lQKIkpCi12Btkl-8fP8QgTPVZlDEgQnU01UwaITlPn_t4oofTVY6fIVylajnzIzT",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 32.685348,
                    "lng": -85.422907
                },
                "viewport": {
                    "northeast": {
                        "lat": 32.68667057989272,
                        "lng": -85.42166462010726
                    },
                    "southwest": {
                        "lat": 32.683970920

{
    "html_attributions": [],
    "next_page_token": "Aap_uEDj-Asl8_18dqvSktnZeU2eugSeNXa56LtlejlgOkLiMfY15TKIvZqUulaC3eVZLFOaqUaCml5MFQ2T8t34lvN0rokQQtaJ3SJxx7HZHWezNINmRxkbvbBo9bnURfraykB75cfpyxLrwoyXy6-D0pRwwO8NhpUlaswnz_MJSlMFZ0ijJnEGcDcSC3c3XkMpu-q0ALkycFcU9reIKzHM2VBrYwuwiuc8X9IUFVC-JV1XHaMa7jPVHv6NRIgwoAF3OR3B0WX6ImhTuZOoD1KPVIwkKIIjjGAI4uF80IibhmmS8mogt1GcRzUCeHo2V9CnpCYnVxvgkOTLjHKI9lVjjLxBbRodTIttmDULYw93Q87hxd3znmpxoIqJtWVuUl-NK1c42-wsEP3rhbhj4bs520QDDcPjlpWFY96Tw6VyscG4ZTlz_Wcre-_FYgnWxOKl",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 14.8995932,
                    "lng": -24.4981834
                },
                "viewport": {
                    "northeast": {
                        "lat": 14.90095017989272,
                        "lng": -24.49681982010728
                    },
                    "southwest": {
                        "lat": 14.8982505

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 31.797713,
                    "lng": 35.235868
                },
                "viewport": {
                    "northeast": {
                        "lat": 31.79944137989272,
                        "lng": 35.23754097989272
                    },
                    "southwest": {
                        "lat": 31.79674172010727,
                        "lng": 35.23484132010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "Dan Jerusalem Hotel",
            "opening_hours": {
                "open_now": true
            },

{
    "html_attributions": [],
    "next_page_token": "Aap_uED3_Q0cjot1So1albUzpx7U4ve83y_Oe5rXMwiY-5cADFtyHQiXBD7tCkV03JpANogURg-dwtoyGVfT_NpuIiNPcFtdrSSQtohjbcgd0Hq0eU7XPuyT9A2KS0x-BLy5MzaHpCi9L4KspsXxyoPTNkxa37DTEHBXQz7S8-wmunmKbOiin7sxeNO-O2DMf0tmLJdYjk3Zew2sc5Z0zh0PXqfqvY5N5LY95Nlvw_Ho0U1nbL9eLofqiEe-V2KWsFKnKs5hNDhjpbBMkKgV4Hwa-8r1Gd0pM3cYurxxIcu8A3uMhiUx0DtVuxGrJ52lkQY9JP6WU-uC-bUrDTPba1Slc8EoVLmsikfDknhFXvbgWdofkwlYC6tJqrpsxgzpri_aNeghyq3szabvbO2T5XAMD6kdcajnAytTORTB5DQg383oCP6UcNJWe1IQRX_sKFbB",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 37.2118357,
                    "lng": -113.2702527
                },
                "viewport": {
                    "northeast": {
                        "lat": 37.21289187989272,
                        "lng": -113.2687988701073
                    },
                    "southwest": {
                        "lat": 37.210192

{
    "html_attributions": [],
    "next_page_token": "Aap_uEAFWEja7ty12j53WA0LDmLnTZRP2QBeKlyx58zaz45pR-73S9Dq51tPQxiDtwsqTH9CiJxcncWJfPd_2i55SMSsp_X1em4LhJkZ82VLH7B2kQiF1uc9LVTwAFRSnagLDNhWBPeLSwCvXGYnEG0PJLneiIwcIQL5i0a-JuahrfuO3zuQ0SIQlBem0CECIETusoEko9ljPqcZRw68hEeWQz854aMfFJ8XzwqXHdae72fJtsaFwUlPLzJJpJFRG8cUZS_2GCT-r5hscJGWWGE7DN9IcfXT6waYuax1zN72MnfSxyUGHTpqj6RKL_ehE-Cz0w7PIeN0XxHtkedWMF7BrV48jc49JxT3C8of-U5C4LwX34o2UC-kx0VuVwcWqHyc8mXO_3ohzm5TKVt7PyBYT5PSSdk_6_WCV0Y5sTuGCVppmdHDq7mbT0hXiNAJCbWe",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 20.6016368,
                    "lng": -105.2378726
                },
                "viewport": {
                    "northeast": {
                        "lat": 20.60298987989272,
                        "lng": -105.2365032701073
                    },
                    "southwest": {
                        "lat": 20.600290

{
    "html_attributions": [],
    "next_page_token": "Aap_uEBGlplShKsADY4zrZpRuos8cgt5glvYPZa52p7PIKYLYOdvej-D9ROmf_e3Myr57J9W6Z3Qim6-jUNA49uYzuDc5XB0wEddhfvWA8wT2evk058Ua8-v-HeluaCP3shxSPW1bW2q9Jh68x0p4uRQHnjjlyRXjXCb6CYnRKoe-w8Vu7elL1dypLI4WB5dPRBY7cCU1B6c2K5ZWomb5Y3t7PTXwf4KDZwNYZFLsEtQzaDCCYGZNK0Ru8dZvCGRXYFVnlb3nsm8fSNt5qjeBFf5OzeoFrLHTLgd5MDDtCYQzfjr-LJkBS3xnVVjNTxXUf7QxvLRY_ohcXZQWhRUM4g-I6CgLx50zcWHaLGht1kLGvx7y3k2x6tu8xFLAAzaPtALXLgMDNUffbiGLljRRHKOz3Sl5bIocLAnPjybXTjqun6rq31RF96_h3pnOp1b8Dy3",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 46.1259206,
                    "lng": 3.4174397
                },
                "viewport": {
                    "northeast": {
                        "lat": 46.12726017989272,
                        "lng": 3.418741729892722
                    },
                    "southwest": {
                        "lat": 46.1245605201

{
    "html_attributions": [],
    "next_page_token": "Aap_uEBV98UHl6Vxs8pk8U_fGLfwOfCARhH_hWYSSP8-UoNm_vSyelJutkCepACbtcLxiyReS3ZOoSCrlNBvBAYY0Hyjibpt6gHIcaRpICZ-EsAGNBZW-Zn_3b3dwIViTRPAL_GXq8HIecVCEoRPTFYd1_gjAjoK2dIwVvwjgu4cX_nKMJeB_1-wUeM1gn_82olpFTO2U3Mxc-J69lD1OOBbC4PYGg89QkVmaVe1ngbNZuE_l8WQvvMbwvoJ695GBSX2YS82Lp3Vg3bPodWvP9CDRpuqQfYAbEUmmhLANwE3jHXxzPUNP_9w7ujmzCC-_oESHNMIDyvjexQ2Gg2qlRaCTL1ky0joR9K0jvGpch1TSqpApYWLEH1nVLRuofMsKtC-QEZOXfke2065zcYlaUTX4bsAe6M_zj6FKYWc-ADJXd9beESnXAAh0UxvAfMiOljw",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 26.9062286,
                    "lng": -101.4295564
                },
                "viewport": {
                    "northeast": {
                        "lat": 26.90753627989272,
                        "lng": -101.4284937701073
                    },
                    "southwest": {
                        "lat": 26.904836

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Temperature,Wind Speed,Hotel Name
0,35,Tupelo,0,US,1651163443,37,34.2576,-88.7034,73.22,5.75,"Holiday Inn Express & Suites Fulton, an IHG Hotel"
1,55,Russell,0,US,1651163450,38,32.3502,-85.1999,78.08,8.05,Auburn Marriott Opelika Resort & Spa at Grand ...
2,128,São Filipe,0,CV,1651163476,71,14.8961,-24.4956,71.98,9.73,Hotel Xaguate
3,226,Kiryat Gat,0,IL,1651163514,44,31.6100,34.7642,73.40,6.82,Dan Jerusalem Hotel
4,270,Saint George,0,US,1651163273,18,37.1041,-113.5841,76.24,5.01,Best Western Plus Zion West Hotel
5,315,Ixtapa,0,MX,1651163286,57,20.7000,-105.2000,78.03,3.44,Playa Los Arcos Hotel Beach Resort & Spa Puert...
6,437,Saint-François,0,FR,1651163571,37,46.4154,3.9054,71.04,9.22,VICHY CÉLESTINS SPA HÔTEL . Vichy - France
7,489,Castaños,0,MX,1651163607,58,26.7833,-101.4167,76.35,0.00,"Holiday Inn Monclova, an IHG Hotel"


In [30]:
hotel_df

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Temperature,Wind Speed,Hotel Name
0,35,Tupelo,0,US,1651163443,37,34.2576,-88.7034,73.22,5.75,"Holiday Inn Express & Suites Fulton, an IHG Hotel"
1,55,Russell,0,US,1651163450,38,32.3502,-85.1999,78.08,8.05,Auburn Marriott Opelika Resort & Spa at Grand ...
2,128,São Filipe,0,CV,1651163476,71,14.8961,-24.4956,71.98,9.73,Hotel Xaguate
3,226,Kiryat Gat,0,IL,1651163514,44,31.6100,34.7642,73.40,6.82,Dan Jerusalem Hotel
4,270,Saint George,0,US,1651163273,18,37.1041,-113.5841,76.24,5.01,Best Western Plus Zion West Hotel
5,315,Ixtapa,0,MX,1651163286,57,20.7000,-105.2000,78.03,3.44,Playa Los Arcos Hotel Beach Resort & Spa Puert...
6,437,Saint-François,0,FR,1651163571,37,46.4154,3.9054,71.04,9.22,VICHY CÉLESTINS SPA HÔTEL . Vichy - France
7,489,Castaños,0,MX,1651163607,58,26.7833,-101.4167,76.35,0.00,"Holiday Inn Monclova, an IHG Hotel"


In [33]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [34]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)

# Display Map
fig


Figure(layout=FigureLayout(height='420px'))